In [14]:
pip install ollama


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openai
import json
# OpenAI Chat Completion call
import ollama

# Replace with your actual OpenAI API key
# Load OCR text from file
with open("ocr_output.json", "r") as f:
    ocr_data = json.load(f)

# Optional: convert OCR lines to raw text string (flatten all blocks' words)
def extract_raw_text(ocr_data):
    raw_lines = []
    for block in ocr_data['pages'][0]['blocks']:
        for line in block.get('lines', []):
            words = [word['value'] for word in line.get('words', [])]
            if words:
                raw_lines.append(" ".join(words))
    return "\n".join(raw_lines)

ocr_text = extract_raw_text(ocr_data)

system_prompt = """
You are an intelligent parser that extracts structured invoice data from raw OCR text and returns it as JSON.

Your responsibilities:
1. Parse all relevant fields:
   - Vendor details (name, address, contact, GSTIN)
   - Invoice number and date
   - Customer billing and shipping info
   - Bank account details (bank name, account number, IFSC, UPI ID if available)
   - Final invoice totals (subtotal, tax breakdown, grand total)
   - Line items with full breakdown

2. Do not miss any line items. Each must include:
   - description
   - hsn
   - uom
   - quantity
   - rate
   - taxable_value
   - cgst (rate, amount)
   - sgst (rate, amount)

3. All tax amounts (CGST, SGST, IGST) and totals must match what's present in the text.

Return only valid JSON, following this structure exactly:
{
  "vendor": {
    "name": ...,
    "address": ...,
    "contact": [...],
    "gstin": ...
  },
  "invoice": {
    "number": ...,
    "date": ...
  },
  "bill_to": {
    "name": ...,
    "address": ...
  },
  "ship_to": {
    "name": ...,
    "address": ...
  },
  "bank_details": {
    "bank_name": ...,
    "account_number": ...,
    "ifsc": ...,
    "upi_id": ...
  },
  "line_items": [
    {
      "description": ...,
      "hsn": ...,
      "uom": ...,
      "quantity": ...,
      "rate": ...,
      "taxable_value": ...,
      "cgst": {
        "rate": ...,
        "amount": ...
      },
      "sgst": {
        "rate": ...,
        "amount": ...
      }
    }
  ],
  "total": {
    "subtotal": ...,
    "cgst_total": ...,
    "sgst_total": ...,
    "igst_total": ...,
    "grand_total": ...
  }
}

Only output valid JSON. Do not add explanations.
"""

user_prompt = f"""
Extract structured invoice data from the following raw OCR text:

{ocr_text}

Ensure that:
- All line items are included
- Final tax amounts (CGST, SGST, IGST) and grand total are captured
- Bank details (bank name, account number, IFSC, UPI ID if present) are extracted

Return only valid JSON as per the above format.
"""



response = ollama.chat(
    model="llama3.1",  # replace with the Ollama model name you want, e.g., "llama3", "mistral", etc.
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    options={
        "temperature": 0.2
    }
)
print("Model response received. Parsing JSON...", response)
# Parse and print JSON output
parsed_json = response['message']['content']

# Try to parse JSON and save to file
try:
    # First attempt to parse the JSON
    structured_data = json.loads(parsed_json)
    
    # Save to file
    with open("response.json", "w", encoding="utf-8") as f:
        json.dump(structured_data, f, ensure_ascii=False, indent=2)
    print("JSON successfully saved to response.json")
    
except json.JSONDecodeError as e:
    print(f"Failed to parse JSON: {e}")
    print("Raw model response:")
    print(parsed_json)
    
    # Save the raw response anyway for debugging
    with open("response_raw.txt", "w", encoding="utf-8") as f:
        f.write(parsed_json)
    print("Saved raw response to response_raw.txt")


Model response received. Parsing JSON... model='llama3.1' created_at='2025-08-14T15:59:54.33263Z' done=True done_reason='stop' total_duration=39392390459 load_duration=42846125 prompt_eval_count=1456 prompt_eval_duration=2808517458 eval_count=1430 eval_duration=36539613500 message=Message(role='assistant', content='{\n  "vendor": {\n    "name": "RAJ SUPER WHOLESALE BAZAR",\n    "address": "45,AMBA PRASAD TIWARI MARG, DAULATGANJ UAIN-MP-456001",\n    "contact": ["0734-4060723", "9993736333"],\n    "gstin": "23CTOPS449201ZX"\n  },\n  "invoice": {\n    "number": "CN3-2254",\n    "date": "27-Feb-2019"\n  },\n  "bill_to": {\n    "name": "RAJ DATA PROCESSORS",\n    "address": ""\n  },\n  "ship_to": {\n    "name": "RAJ DATA PROCESSORS",\n    "address": "45, DAULATGANJ, UJJAIN"\n  },\n  "bank_details": {\n    "bank_name": "BANK OF INDIA",\n    "account_number": "9100123456456",\n    "ifsc": "BKID00001901",\n    "upi_id": ""\n  },\n  "line_items": [\n    {\n      "description": "SWADIST SOYA OI